# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [2]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c
set.seed(123)
simulate_data <- function(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
                            age = runif(sample_size, age_lo, age_hi)
                            epsilon_x = rnorm(sample_size, 0, sd = sd_x)
                            x = age*beta_xa+ beta_x0 +epsilon_x

                            epsilon_c = rnorm(sample_size, 0, sd = sd_c)
                            c = age*beta_ca + beta_c0 + epsilon_c

                            data.frame(age = age, x = x, c = c)
      return(data.frame(age=age,x=x,c=c)) # it's actually bad form to have a variable named "c" in R, my bad...
}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)

,age,x,c
,<dbl>,<dbl>,<dbl>
1,114.50930,64.92058,168.56524
2,174.59662,80.87097,215.04588
3,129.07723,57.39509,141.49900
4,185.96209,156.41116,73.05766
5,192.85607,80.13949,154.13139
6,85.46678,113.55692,54.53982


---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [4]:
install.packages('lme4')
library(lme4)
library(tidyverse)
install.packages('mediation')
library(mediation)
set.seed(123)
run_analysis <- function(dat) {

  model_m <- lm(x ~ age, data = dat)
  model_y <- lm(c ~ x + age, data = dat)

  fitMed <- mediate(model.m = model_m, model.y = model_y, treat = 'age', mediator = 'x')


  return(c(ACME = fitMed$d0,
           ACME_p = fitMed$d0.p,
           ADE = fitMed$z0,
           ADE_p = fitMed$z0.p))
}
results <- run_analysis(dat)
print(results)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘Rdpack’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’


Loading required package: Matrix

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘zoo’,

       ACME      ACME_p         ADE       ADE_p 
-0.06764012  0.36600000  0.79213635  0.00200000 


---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [5]:
set.seed(123)
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations*4), nrow=num_simulations, ncol=4)

    # Start simulating
    for (i in 1:num_simulations) {
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      simouts[i,] <- run_analysis(dat)
    }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov =  mean(simouts[, 2] < alpha)
    ADE_cov =  mean(simouts[,4] <alpha )

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}
results

ACME      ACME_p         ADE       ADE_p 
-0.06764012  0.36600000  0.79213635  0.00200000

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

In [7]:
set.seed(123)
results = repeat_analysis(num_simulations = 10, alpha = 0.01, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
results[1:4]

$ACME_cov
[1] 0

$ADE_cov
[1] 0.7

$<NA>
NULL

$<NA>
NULL

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [10]:
sample_sizes <- c(50, 75, 100, 125, 150)
power_df <- data.frame(
  sample_size = sample_sizes,
  ACME_cov = NA,
  ADE_cov = NA
)

set.seed(123)

for (i in seq_along(sample_sizes)) {
  size <- sample_sizes[i]

  result <- repeat_analysis(num_simulations = 10, alpha = 0.01, sample_size = size,
  age_lo = age_lo, age_hi = age_hi, beta_xa = beta_xa, beta_x0 = beta_x0, sd_x = sd_x,
  beta_ca = beta_ca, beta_cx = beta_cx, beta_c0 = beta_c0,sd_c = sd_c
  )

  power_df$ACME_cov[i] <- result$ACME_cov
  power_df$ADE_cov[i] <- result$ADE_cov
}

Print your results.

In [11]:
print(power_df)

  sample_size ACME_cov ADE_cov
1          50      0.1     0.5
2          75      0.0     0.4
3         100      0.0     0.9
4         125      0.0     0.9
5         150      0.0     1.0


## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> Since power is generally considered high at 80%, the results above show that power can be high starting at 100 participants (90%). However, there is low power to detect mediation.
>

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> There was a really high standard deviation which may cause a lot of noise with little participants and the alpha level of 0.01 is strict compared to 0.05.

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*